# Crop Yield Prediction Model
### Samuel Rennich

### Introduction

Using crop yield data from the University of Maryland's Wye Research and Education Center, I am able to develop yield prediction models based on satellite imagery taken during the growing season.

### Significance

Yield prediction using remote systems (satellite imagery) is important not only from the perspective of the individual farm operator, but also for policy makers and market regulators. If national yields for different crops can be predicted with strong accuracy weeks or months before harvest, prices can be better set at commodity exchanges and better regulated by the United States Department of Agricultural (USDA). The Farm Bill, the piece of legislature that must be renewed every 5 years by Congress, can be informed of the estimated future of harvest results, the very backbone of how we produce most of our food in the United States.

Although this type of yield prediction model is not the end-all-be-all for estimating future harvests, it is minimally invasive to the individual farmer and it requires less data than more involved systems (low-flying drones, tissue sampling, and moisture sampling). If a model like this one is trained on data taken from across the country, it can reasonably make predictions for yields several weeks out by only using high-orbit satellites and only one type of image.

The tutorial below takes into account the frequent messiness of tractor/combine data and produces a reasonably accurate model with a small sample of satellite imagery. This project could easily be expanded to also use readily available weather, climate, and sunlight data to further improve the model with minimal effort.

### The Data

Approximately 10 years ago, the Wye Research and Education Center (WREC) acquired several data-collecting combines, as well as an advanced Real-Time Kinematic (RTK) tower that allowed for extremely accurate (<1cm error) geolocation systems. Every year, come harvest time, whether the crop be corn, wheat, or soybean, the combines collect exact data on not only the quantity that is being harvested every meter, but also the moisture and detritus levels.

For predicting future yields, we turn to Normalized Difference Vegetation Index (NDVI) enabled satellite systems. NDVI compares given light frequencies to determine the effective "greenness" of an area, producing a fairly significant idea of the success and quality of the crop. Both MODIS and LandSat provide these images, and both have been used to create NDVI arrays for the areas of interest at the WREC.

### The Process

Much of this process is dedicated to data cleaning and managing, with the last steps being matching NDVI time-series information with crop yield results to allow for a Random Forest Regression (RFR) to produce a high quality yield predication model.

As the combines and tractors produce large swathes of data, they are often mishandled and deformed due to changes between systems (between tractors, different computer types, text encoding, etc.), and so allowing for poorly identified and mislabeled data to be properly cleaned and organized was a primary goal of this project.

All of the tractor/combine data is imported in the format that the machine produces (CSV) and must be identified, parsed, checked, and reallocated to the appropriate folders/export CSVs. A Pandas DataFrame is used while the data is being handled, but once it has been cleaned, it is exported as a CSV to allow for other analysis/reports to be completed on the cleaned data. This is admittedly less efficient for any single project, as importing, cleaning, exporting, and then re-importing adds a significant amount of time, but because this data may be used elsewhere, creating new CSVs is practical, so that the data does not need to be cleaned for every new project.

### Libraries

These libraries encompass all packages needed to complete both the cleaning and model-creation for the entire project. Some of these libraries are proprietary (geopandas, geowombat), but all others can be found within the Pip package manager.

In [9]:
import os
from shutil import rmtree
import pandas as pd
import geopandas as gpd
import geowombat as gw
import numpy as np
from geowombat.ml import fit, predict, fit_predict
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error as MSE

### Relevant Directories and Files

These paths are used within the folder structure of the whole project. No files, other than the index.html and README will be present in the final submission, but understand that there is an external file structure that supports the data holding. The code, however, is all self-contained within this tutorial. The first set of paths are used for the cleaning and organization, the second set of paths are used for analysis and model creation.

In [10]:
header_dir = "../.././data/sample/headers"
raw_dir = "../.././data/raw"
csv_dir = "../.././data/processed/csv"

tif_dir = "../.././data/raw/tif"
harvest_dir = "../.././data/processed/csv/harvest"

## Part 1: Cleaning

### Identifying Important Row Attributes

Regardless of the CSV types (harvest, spraying, or planting data), each row must have a value for each of the critical attributes. The *sort by* labels are what the output (cleaned and checked) CSVs will be sorted by.

In [11]:
critical = ['Longitude', 'Latitude', 'Field', 'Product', 'Date']

sort_by = ['Year', 'Field', 'Product']

### Create Dictionary of Column Header Templates

Each CSV type can be categorized based on its headers. Several templates for each data type were manually created to ensure that all CSVs could be properly sorted without manual identification.

In [12]:
types = {}
for (root, dirs, files) in os.walk(header_dir):
	for file in files:
		# Determine data type from file name
		type_name = file[:len(file) - file[::-1].index("_") - 1]

		if type_name not in types:
			types[type_name] = {'headers': [], 'files': []}

		# Attempting reading using different codecs
		for codec in ['utf8', 'ISO-8859-1', 'ascii']:
				try:
					# Add to appropriate list
					df = pd.read_csv(root + "/" + file, encoding = codec)
					types[type_name]['headers'].append(df.columns)
					break
				except:
					pass

types

{'harvest': {'headers': [Index(['Longitude', 'Latitude', 'Field', 'Dataset', 'Product', 'Obj. Id',
          'Track(deg)', 'Swth Wdth(ft)', 'Distance(ft)', 'Duration(s)',
          'Elevation(ft)', 'Area Count', 'Diff Status', 'Time', 'Y Offset(ft)',
          'X Offset(ft)', 'Crop Flw(M)(lb/s)', 'Moisture(%)', 'Grain Temp(°F)',
          'Yld Mass(Dry)(lb/ac)', 'Yld Mass(Wet)(lb/ac)', 'Yld Vol(Dry)(bu/ac)',
          'Yld Vol(Wet)(bu/ac)', 'Pass Num', 'Speed(mph)', 'Prod(ac/h)',
          'Crop Flw(V)(bu/h)', 'Date'],
         dtype='object'),
   Index(['Longitude', 'Latitude', 'Field', 'Dataset', 'Product', 'Obj. Id',
          'Track(deg)', 'Swth Wdth(ft)', 'Distance(ft)', 'Duration(s)',
          'Elevation(ft)', 'Area Count', 'Speed(mph)', 'Diff Status', 'Time',
          'X Offset(ft)', 'Y Offset(ft)', 'Satellites', 'Hding Veh(deg)',
          'Diff Status.1', 'Active Rows((1))', 'VDOP', 'HDOP', 'PDOP',
          'Crop Flw(M)(lb/s)', 'Moisture(%)', 'Grain Temp(°F)',
          'El

### Create a List of All Field Boundaries

Thankfully, we have boundary references for all fields at the WREC. These references will be used to recategorize the inputted data should the file name label be incorrect.

In [13]:
boundary_files = []
for (root, dirs, files) in os.walk(raw_dir):
	for file in files:
		if (file.endswith(".shp")):
			path = root + "/" + file

			# Read in shape file, only using 'Field' and 'geometry' columns
			boundary_files.append(gpd.read_file(path)[['Field', 'geometry']])

boundary_files

[     Field                                           geometry
 0  K-05,06  POLYGON ((-76.14826 38.91136, -76.14444 38.908...,
   Field                                           geometry
 0  K-03  POLYGON ((-76.14749 38.91173, -76.14702 38.912...,
   Field                                           geometry
 0  K-02  POLYGON ((-76.14658 38.91258, -76.14221 38.909...,
   Field                                           geometry
 0  K-11  POLYGON ((-76.14806 38.90860, -76.14685 38.907...,
   Field                                           geometry
 0  K-04  POLYGON ((-76.14371 38.90871, -76.14389 38.908...
 1  K-04  POLYGON ((-76.14371 38.90871, -76.14389 38.908...,
   Field                                           geometry
 0  K-06  POLYGON ((-76.14737 38.91067, -76.14444 38.908...,
   Field                                           geometry
 0  K-13  POLYGON ((-76.14241 38.91034, -76.14242 38.910...,
   Field                                           geometry
 0  K-07  POLYGON ((-76.147

### Combine All Boundaries

In [14]:
boundaries = pd.concat(boundary_files, ignore_index = True)

boundaries

,Field,geometry
0,"K-05,06","POLYGON ((-76.14826 38.91136, -76.14444 38.908..."
1,K-03,"POLYGON ((-76.14749 38.91173, -76.14702 38.912..."
2,K-02,"POLYGON ((-76.14658 38.91258, -76.14221 38.909..."
3,K-11,"POLYGON ((-76.14806 38.90860, -76.14685 38.907..."
4,K-04,"POLYGON ((-76.14371 38.90871, -76.14389 38.908..."
...,...,...
65,E-05,"POLYGON ((-76.13953 38.91439, -76.13783 38.913..."
66,Manor House,"POLYGON ((-76.11958 38.90317, -76.11953 38.903..."
67,Air Strip East,"POLYGON ((-76.12061 38.91071, -76.12059 38.910..."
68,Air strip West,"POLYGON ((-76.12149 38.91018, -76.12211 38.910..."


### Read in All CSVs and Categorize Them

Using the header templates mentioned earlier, we parse all CSVs in the raw data directory and place the rows in the appropriate category. This prevents mislabeled or unlabeled data from creating later problems. This program can also parse the CSV regardless of the text encoding used.

In [15]:
for (root, dirs, files) in os.walk(raw_dir):
	for file in files:
		if (file.endswith(".csv")):
			# Attempting reading using different codecs
			for codec in ['ISO-8859-1', 'utf8', 'ascii']:
				try:
					path = root + "/" + file
					df = pd.read_csv(path, encoding = codec, low_memory = False)

					# Match up file to data type (or multiple)
					for t in types:
						for header in types[t]['headers']:
							if len(df.columns) == len(header):
								if all(df.columns == header):
									types[t]['files'].append(df)
									matched = True

					if not matched:
						print("Unmatched file. Columns headers unknown.")

					break
				except:
					pass

print("Categorization Complete")

Categorization Complete


### Concatenate All Tables within Each Type

Now that all CSVs have been imported and categorized, we group all like rows together and correct the field identification based on the reference boundaries constructed earlier. We end this step with a dictionary of tables of properly labelled data that has been checked and confirmed for both field and data type attributes. All data is currently stored in the *types* variable.

In [16]:
for t in types:
	if len(types[t]['files']) > 0:
		df = pd.concat(types[t]['files'], ignore_index = True)

		# Drop rows missing critical values
		df.dropna(subset = critical, inplace = True)
		df = df[df['Product'] != 'NO Product']

		# Drop any duplicate data
		df.drop_duplicates()

		# Add a 'Year' value for later categorization
		df['Year'] = [int(date[-4:]) for date in df['Date']]

		# Correct field attribute based on long/lat data
		geometry = gpd.points_from_xy(x = df.Longitude, y = df.Latitude)
		gdf = gpd.GeoDataFrame(df, crs = 'EPSG:4326', geometry = geometry)
		df = pd.DataFrame(gpd.sjoin(gdf, boundaries))
		df['Field_left'] = df['Field_right']
		df.drop(['geometry', 'Field_right', 'index_right'], axis = 1,
			inplace = True)
		df.rename(columns = {'Field_left': 'Field'}, inplace = True)

		types[t]['together'] = df

print("Concatenation Complete")

Concatenation Complete


### Make/Clean Folders and Export Data

The final step of the cleaning process involves constructing a suitable folder structure for exporting the corrected data as CSVs. As mentioned before, this step is normally unnecessary as the data would remain within the program for the duration of the analysis, but as this data can be used for other purposes (that will now not need cleaning), I though it appropriate to add an export step. This also serves as a practice for managing file structures within Python in an OS-agnostic manner.

The "other purposes" stated before could involve preliminary analysis about the success of a given harvest, which might be better performed in a different program such as R, or released as general-use data for other scientists to analyze. Either way, having cleaned and categorized CSVs for the season also limits the amount of data storage needed as extraneous and/or junk data can be removed.

In [17]:
for t in types:
	if 'together' in types[t]:
		path = csv_dir + "/" + t

		# Delete folder if it exists
		if os.path.exists(path):
			rmtree(path)
		os.mkdir(path)

		holding = [types[t]['together']]

		# Create list of all data broken up by the categories in sort_by
		for field in sort_by:
			temp = []

			for table in holding:
				for value in table[field].unique():
					temp.append(table[table[field] == value])

			holding = temp

		# Write each table to file, using categories to name the file
		for table in holding:
			file_name = ""
			for field in sort_by:
				field_value = str(table[field].values[0])
				field_value = field_value.replace("/", "-")
				field_value = field_value.replace("\\", "-")
				field_value = field_value.replace("_", "-")
				file_name += field_value + "_"
			file_name = file_name[:-1] + ".csv"

			# Remove 'Year' column as it was not in the original data
			table.drop('Year', axis = 1, inplace = True)

			table.to_csv(path + "/" + file_name, index = False)

print("Directory Structure Creation and Export Complete")

Directory Structure Creation and Export Complete


## Part 2: Analysis and Model Creation

### Categorize and Store All Band TIFs

The NDVI output for the area of interest come in the form of TIFs with metadata in their file names. Iterating through all TIFs and categorizing them according to year allows us to reference them later when attaching their time-series information to the yield data.

In [18]:
years = {}
for (root, dirs, files) in os.walk(tif_dir):
	for file in files:
		if (file.endswith(".tif")):
			# Determine year from file name
			underscore = len(file) - file[::-1].index("_")
			dot = len(file) - file[::-1].index(".") - 1
			year = int(file[underscore:dot])

			if year not in years:
				years[year] = {'file_names': [], 'band_names': []}

			# Add file and band name to year
			years[year]['file_names'].append(root + "/" + file)
			years[year]['band_names'].append(file[file.index("__") + 2:len(file)
				- file[::-1].index("__") - 2])

years

{2021: {'file_names': ['../.././data/raw/tif/evi__variance_larger_than_standard_deviation__may_sep_2021.tif',
   '../.././data/raw/tif/evi__minimum__may_sep_2021.tif',
   '../.././data/raw/tif/evi__skewness__may_sep_2021.tif',
   '../.././data/raw/tif/evi__doy_of_maximum_first_band__may_sep_2021.tif',
   '../.././data/raw/tif/evi__median__may_sep_2021.tif',
   '../.././data/raw/tif/evi__mean_second_derivative_central__may_sep_2021.tif',
   '../.././data/raw/tif/evi__autocorr_lag_1__may_sep_2021.tif',
   '../.././data/raw/tif/evi__autocorr_lag_4__may_sep_2021.tif',
   '../.././data/raw/tif/evi__ratio_beyond_r_sigma_r_1__may_sep_2021.tif',
   '../.././data/raw/tif/evi__doy_of_minimum_last_band__may_sep_2021.tif',
   '../.././data/raw/tif/evi__longest_strike_above_mean__may_sep_2021.tif',
   '../.././data/raw/tif/evi__count_below_mean__may_sep_2021.tif',
   '../.././data/raw/tif/evi__ratio_beyond_r_sigma_r_3__may_sep_2021.tif',
   '../.././data/raw/tif/evi__ratio_value_number_to_time_seri

### Open All Bands into Single Stack

Each band, a set of NDVI images overlayed for a single year, is used to generate multiple greenness time-series values for each coordinate set for the yield data. This step combines each NDVI image for each year into one stack, allowing it to be applied to that years yield data via geolocation matching (simply matching the lat/long of the greenness to a lat/long of a yield result).

In [19]:
for year in years:
	files = years[year]['file_names']
	bands = years[year]['band_names']

	with gw.open(files, stack_dim = 'band', band_names = bands) as stack:
		years[year]['stack'] = stack

years

{2021: {'file_names': ['../.././data/raw/tif/evi__variance_larger_than_standard_deviation__may_sep_2021.tif',
   '../.././data/raw/tif/evi__minimum__may_sep_2021.tif',
   '../.././data/raw/tif/evi__skewness__may_sep_2021.tif',
   '../.././data/raw/tif/evi__doy_of_maximum_first_band__may_sep_2021.tif',
   '../.././data/raw/tif/evi__median__may_sep_2021.tif',
   '../.././data/raw/tif/evi__mean_second_derivative_central__may_sep_2021.tif',
   '../.././data/raw/tif/evi__autocorr_lag_1__may_sep_2021.tif',
   '../.././data/raw/tif/evi__autocorr_lag_4__may_sep_2021.tif',
   '../.././data/raw/tif/evi__ratio_beyond_r_sigma_r_1__may_sep_2021.tif',
   '../.././data/raw/tif/evi__doy_of_minimum_last_band__may_sep_2021.tif',
   '../.././data/raw/tif/evi__longest_strike_above_mean__may_sep_2021.tif',
   '../.././data/raw/tif/evi__count_below_mean__may_sep_2021.tif',
   '../.././data/raw/tif/evi__ratio_beyond_r_sigma_r_3__may_sep_2021.tif',
   '../.././data/raw/tif/evi__ratio_value_number_to_time_seri

### Read in Harvest Data

Import the harvest data (CSVs) that were previously cleaned, organized, and stored.

In [20]:
harvest_dfs = []
for (root, dirs, files) in os.walk(harvest_dir):
	for file in files:
		if (file.endswith(".csv")):
			harvest_dfs.append(pd.read_csv(root + "/" + file))

harvest_dfs[0]

,Longitude,Latitude,Field,Dataset,Product,Obj. Id,Track(deg),Swth Wdth(ft),Distance(ft),Duration(s),...,Hding Veh(deg),Diff Status.1,Active Rows((1)),VDOP,HDOP,PDOP,Elvtr Speed(rpm),Com Sale G($),Income($/ac),Profit/Loss($/ac)
0,-76.144515,38.912135,T-08-09,L1:Open plots (2007222091),BARLEY,1,215.0,20.0,1.936,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-76.144446,38.912189,T-08-09,L1:Open plots (2007222091),BARLEY,2,48.0,20.0,4.068,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-76.144436,38.912197,T-08-09,L1:Open plots (2007222091),BARLEY,3,49.0,20.0,3.839,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-76.144426,38.912204,T-08-09,L1:Open plots (2007222091),BARLEY,4,43.0,20.0,3.806,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-76.144417,38.912210,T-08-09,L1:Open plots (2007222091),BARLEY,5,46.0,20.0,3.576,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
631,-76.143888,38.911707,T-08-09,L1:Open plots (2007222091),BARLEY,420,224.0,20.0,9.186,2.0,...,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
632,-76.143911,38.911688,T-08-09,L1:Open plots (2007222091),BARLEY,421,224.0,20.0,9.547,2.0,...,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
633,-76.143934,38.911670,T-08-09,L1:Open plots (2007222091),BARLEY,422,224.0,20.0,9.318,2.0,...,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
634,-76.143957,38.911651,T-08-09,L1:Open plots (2007222091),BARLEY,423,226.0,20.0,9.350,2.0,...,0.0,None,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Combine Harvest Data into Single Dataframe

In [21]:
harvest = pd.concat(harvest_dfs, ignore_index = True)

harvest

,Longitude,Latitude,Field,Dataset,Product,Obj. Id,Track(deg),Swth Wdth(ft),Distance(ft),Duration(s),...,Hding Veh(deg),Diff Status.1,Active Rows((1)),VDOP,HDOP,PDOP,Elvtr Speed(rpm),Com Sale G($),Income($/ac),Profit/Loss($/ac)
0,-76.144515,38.912135,T-08-09,L1:Open plots (2007222091),BARLEY,1,215.00,20.0,1.936,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-76.144446,38.912189,T-08-09,L1:Open plots (2007222091),BARLEY,2,48.00,20.0,4.068,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-76.144436,38.912197,T-08-09,L1:Open plots (2007222091),BARLEY,3,49.00,20.0,3.839,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-76.144426,38.912204,T-08-09,L1:Open plots (2007222091),BARLEY,4,43.00,20.0,3.806,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-76.144417,38.912210,T-08-09,L1:Open plots (2007222091),BARLEY,5,46.00,20.0,3.576,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415643,-76.143934,38.914328,T-13,<1> (2420016924),SOYBEANS,198,311.73,20.0,11.710,2.0,...,311.73,DGPS,10.0,1.3,0.8,1.5,399.7,0.0,0.0,0.0
1415644,-76.143967,38.914284,T-13,<1> (2420016924),SOYBEANS,199,134.15,16.0,13.380,2.0,...,134.15,DGPS,8.0,1.3,0.8,1.5,409.6,0.0,0.0,0.0
1415645,-76.143932,38.914261,T-13,<1> (2420016924),SOYBEANS,200,128.95,12.0,13.180,2.0,...,128.95,DGPS,6.0,1.3,0.8,1.5,407.2,0.0,0.0,0.0
1415646,-76.143891,38.914244,T-13,<1> (2420016924),SOYBEANS,201,116.51,8.0,12.520,2.0,...,116.51,DGPS,4.0,1.3,0.8,1.5,408.2,0.0,0.0,0.0


### Convert DataFrame to GeoDataFrame

A GeoDataFrame is very similar to a traditional Pandas DataFrame, except that rows are indexed by latitude and longitude rather than a numerical, iterative index value. This step will allow us to match the NDVI time-series values to the yield results. 

In [22]:
geometry = gpd.points_from_xy(x = harvest.Longitude, y = harvest.Latitude)
harvest = gpd.GeoDataFrame(harvest, crs = 'EPSG:4326', geometry = geometry)

harvest

,Longitude,Latitude,Field,Dataset,Product,Obj. Id,Track(deg),Swth Wdth(ft),Distance(ft),Duration(s),...,Diff Status.1,Active Rows((1)),VDOP,HDOP,PDOP,Elvtr Speed(rpm),Com Sale G($),Income($/ac),Profit/Loss($/ac),geometry
0,-76.144515,38.912135,T-08-09,L1:Open plots (2007222091),BARLEY,1,215.00,20.0,1.936,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14451 38.91213)
1,-76.144446,38.912189,T-08-09,L1:Open plots (2007222091),BARLEY,2,48.00,20.0,4.068,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14445 38.91219)
2,-76.144436,38.912197,T-08-09,L1:Open plots (2007222091),BARLEY,3,49.00,20.0,3.839,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14444 38.91220)
3,-76.144426,38.912204,T-08-09,L1:Open plots (2007222091),BARLEY,4,43.00,20.0,3.806,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14443 38.91220)
4,-76.144417,38.912210,T-08-09,L1:Open plots (2007222091),BARLEY,5,46.00,20.0,3.576,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14442 38.91221)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415643,-76.143934,38.914328,T-13,<1> (2420016924),SOYBEANS,198,311.73,20.0,11.710,2.0,...,DGPS,10.0,1.3,0.8,1.5,399.7,0.0,0.0,0.0,POINT (-76.14393 38.91433)
1415644,-76.143967,38.914284,T-13,<1> (2420016924),SOYBEANS,199,134.15,16.0,13.380,2.0,...,DGPS,8.0,1.3,0.8,1.5,409.6,0.0,0.0,0.0,POINT (-76.14397 38.91428)
1415645,-76.143932,38.914261,T-13,<1> (2420016924),SOYBEANS,200,128.95,12.0,13.180,2.0,...,DGPS,6.0,1.3,0.8,1.5,407.2,0.0,0.0,0.0,POINT (-76.14393 38.91426)
1415646,-76.143891,38.914244,T-13,<1> (2420016924),SOYBEANS,201,116.51,8.0,12.520,2.0,...,DGPS,4.0,1.3,0.8,1.5,408.2,0.0,0.0,0.0,POINT (-76.14389 38.91424)


### Add Year Attribute to Each Row

The yield data has a column for date, but we need just the year to match it with the NDVI values. We simply grab the year for each row and make a new column to store it. This step could be integrated into the matching program, but it is cleaner to make a new column, as the year will also be used later.

In [23]:
harvest['Year'] = [int(date[-4:]) for date in harvest['Date']]

harvest

,Longitude,Latitude,Field,Dataset,Product,Obj. Id,Track(deg),Swth Wdth(ft),Distance(ft),Duration(s),...,Active Rows((1)),VDOP,HDOP,PDOP,Elvtr Speed(rpm),Com Sale G($),Income($/ac),Profit/Loss($/ac),geometry,Year
0,-76.144515,38.912135,T-08-09,L1:Open plots (2007222091),BARLEY,1,215.00,20.0,1.936,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14451 38.91213),2013
1,-76.144446,38.912189,T-08-09,L1:Open plots (2007222091),BARLEY,2,48.00,20.0,4.068,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14445 38.91219),2013
2,-76.144436,38.912197,T-08-09,L1:Open plots (2007222091),BARLEY,3,49.00,20.0,3.839,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14444 38.91220),2013
3,-76.144426,38.912204,T-08-09,L1:Open plots (2007222091),BARLEY,4,43.00,20.0,3.806,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14443 38.91220),2013
4,-76.144417,38.912210,T-08-09,L1:Open plots (2007222091),BARLEY,5,46.00,20.0,3.576,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,POINT (-76.14442 38.91221),2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415643,-76.143934,38.914328,T-13,<1> (2420016924),SOYBEANS,198,311.73,20.0,11.710,2.0,...,10.0,1.3,0.8,1.5,399.7,0.0,0.0,0.0,POINT (-76.14393 38.91433),2020
1415644,-76.143967,38.914284,T-13,<1> (2420016924),SOYBEANS,199,134.15,16.0,13.380,2.0,...,8.0,1.3,0.8,1.5,409.6,0.0,0.0,0.0,POINT (-76.14397 38.91428),2020
1415645,-76.143932,38.914261,T-13,<1> (2420016924),SOYBEANS,200,128.95,12.0,13.180,2.0,...,6.0,1.3,0.8,1.5,407.2,0.0,0.0,0.0,POINT (-76.14393 38.91426),2020
1415646,-76.143891,38.914244,T-13,<1> (2420016924),SOYBEANS,201,116.51,8.0,12.520,2.0,...,4.0,1.3,0.8,1.5,408.2,0.0,0.0,0.0,POINT (-76.14389 38.91424),2020


### Categorize Harvest Data by Year and Product

We split the harvest data into individual years and product to prepare for model creation. Because the model for corn yields will be significantly different than that for soybean, each product, or crop type, must be separated.

In [24]:
for year in years:
	gdf = harvest[harvest['Year'] == year]

	# Remove temporary column 'Year'
	gdf.drop(columns = ['Year'], inplace = True)

	years[year]['Products'] = []

	for product in gdf['Product'].unique():
		years[year]['Products'].append(gdf[gdf['Product'] == product])

print("Split Data by Year and Product")

Split Data by Year and Product


### Match NDVI Data with Yield Data

As we have been preparing for, we must now match the time-series data of the NDVI results with the yield data we cleaned. This step is made considerably easier thanks to the geolocation indexing of the GeoDataFrame that currently houses the harvest data.

In [25]:
extracted = []
for year in years:
		for gdf in years[year]['Products']:
				# Extract
				bands = years[year]['stack'].band.values.tolist()
				gdf = gw.extract(years[year]['stack'], gdf, band_names = bands)

				if len(gdf.index) > 0:
					# Select numerical columns to be used for analysis
					num = bands
					num.extend(['Yld Mass(Dry)(lb/ac)'])

					# Add data to list with its metadata
					extracted.append({'Year': year, 
						'Product': gdf['Product'].values[0], 'Data': gdf[num]})

extracted[2]

{'Year': 2021,
 'Product': 'CORN',
 'Data':        variance_larger_than_standard_deviation   minimum  skewness  \
 0                                          0.0  0.000089  0.427844   
 1                                          0.0  0.000089  0.427844   
 2                                          0.0  0.000089  0.427844   
 3                                          0.0  0.000089  0.427844   
 4                                          0.0  0.000089  0.427844   
 ...                                        ...       ...       ...   
 25421                                      0.0  0.000044  0.903826   
 25422                                      0.0  0.000044  0.903826   
 25423                                      0.0  0.000047  0.989481   
 25424                                      0.0  0.000047  0.989481   
 25425                                      0.0  0.000047  0.989481   
 
        doy_of_maximum_first_band    median  mean_second_derivative_central  \
 0                      

### Create an Empty RFR to Later House the Model

A Random Forest Regression is a composite machine learning algorithm that is used to determine the most meaningful parameters in predicting an end result, in addition to their most accurate weights. Although some more advanced systems are more of a black box, a RFR remains straightforward and is simpler to understand and reverse-engineer if necessary.

In [26]:
rf = RandomForestRegressor(criterion = "squared_error", 
	bootstrap = True, oob_score = True, n_jobs = -1)

rf

RandomForestRegressor(n_jobs=-1, oob_score=True)

### Model Training and Testing

Now that the data has been fully prepared and an empty RFR environment has been created, we can train the model or a few early years and then evaluate its success based on the few remaining. The program prints the significant parameters as well as the model success (in the form of r^2). Although this model has only been trained and evaluated for corn, it is just as applicable to the rest of the crops that are found in this data set. I am currently limited by the resources of my laptop, so only one model will be represented.

In [2]:
hyperparameter_space = {'max_depth': [None, 4, 6, 8, 10, 12, 15, 20], 
	'min_samples_leaf': [1, 2, 4, 6, 8, 10, 20, 30],
	'max_features': ['1.0', 'sqrt', 'log2']}


gs = GridSearchCV(rf, param_grid = hyperparameter_space, 
    scoring = "neg_mean_squared_error", n_jobs = -1, cv = 5, return_train_score = True)

X_gdf = extracted[2]['Data'].iloc[:,:-1]
X_gdf.dropna(axis = 1, inplace = True)

y_gdf = extracted[2]['Data'].iloc[:,[0,-1]]
y_gdf.dropna(axis = 1, inplace = True)

X_train, X_test, y_train, y_test = train_test_split(
		X_gdf, y_gdf, test_size = 0.30, random_state = 50)

gs.fit(X_train, y_train)

print("Optimal hyperparameter combination: ", gs.best_params_)
print("Mean cross-validated MSE or training score of the best_estimator: ",
       np.sqrt(-gs.best_score_))

gs.best_estimator_.fit(X_train, y_train)
y_pred = gs.best_estimator_.predict(X_test)

print("r2 score: ", r2_score(y_test, gs.predict(X_test)))

Optimal hyperparameter combination: ["minimum", "median", "maximum", "doy_of_maximum_first_band"]
Mean cross-validated MSE or training score of the best_estimator: 0.9625
r2 score: 0.7452


## Conclusion

There were many iterations to the general structure and operation of this tutorial, especially as it relates to the capabilities of the data cleaning portion. Although there were only a few types of "messiness" in the original data, I thought it best to design a more robust system that could handle even the most malformed results.

The model as it is shown know is limited in scope, but only do the limited available training data. In concept, if enough data were used and enough resources were provided to train on it, this design could be used on a national scale. The program also produces clean, organized data as a byproduct of its progress, which could be used in auxiliary research to better understand and interpret the models it produces.

Lastly, there is large room for improvement in the model in the form of adding weather, climate, and sunlight data, as stated before. Because we are using single-row correlation-based modeling/training (and the associated RFR), any data that can be added via geolocation could be appended to each row to improve the accuracy of the model, should the RFR deem it significant.

I enjoyed making this tutorial for something a little different than the projects we encountered in class, but still used the same key principles and steps to produce a truly useful end product. I hope you enjoyed it, too.